In [1]:
import pandas as pd
import tqdm
from typing import Optional

import re
import nltk

import json

from gensim.models import Word2Vec

In [2]:
# articles
!gdown 1BAIEjxgx-Q0hGV9ByLajBd-5Id2NZQE5

Downloading...
From: https://drive.google.com/uc?id=1BAIEjxgx-Q0hGV9ByLajBd-5Id2NZQE5
To: /content/articles.csv
100% 25.2M/25.2M [00:00<00:00, 143MB/s] 


In [3]:
nltk.download('stopwords')
stopword_set = set(nltk.corpus.stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
articles = pd.read_csv('articles.csv')
articles.head()

,date,title,body,count,url
0,2024-02-23,Зеленский без плана Б и с угрозой паралича эко...,Президент Украины Владимир Зеленский не смог о...,673,/mezhdunarodnaya-panorama/20061391
1,2024-02-23,Как советские летчики впервые долетели напряму...,"24 февраля 1899 года, 125 лет назад, родился в...",179,/infographics/9497
2,2024-02-23,"Провал ""украинизации"" G20 и капризы Запада. За...","Запад стремится наказать всех тех, кто не след...",472,/politika/20062849
3,2024-02-23,Военная операция на Украине. Хроника событий 2...,Госдепартамент США внес в свои санкционные спи...,59,/armiya-i-opk/20054555
4,2024-02-23,Военная операция на Украине. Хроника событий 2...,Минобороны РФ объявило об освобождении населен...,51,/armiya-i-opk/20043925


**отделим тела новостей, оставим только буквы, токенизируем, приведём к нижнему регистру**

In [5]:
corpus = articles.body.values
print(f'len(corpus) = {len(corpus)}, type(corpus) = {type(corpus)}\n')
print(corpus[0], end='\n\n')

TOKEN_PATTERN = re.compile('[а-яА-ЯёЁ]+')


def tokenize(text):
    return re.findall(TOKEN_PATTERN, text.lower())


docs = [tokenize(text) for text in corpus]
print(docs[0])

len(corpus) = 2396, type(corpus) = <class 'numpy.ndarray'>

Президент Украины Владимир Зеленский не смог объяснить, в чем заключается его план Б на тот случай, если Вашингтон не предоставит Киеву новый пакет помощи. У украинских властей нет выбора, проводить или не проводить мобилизацию, так как часть подразделений Вооруженных сил Украины (ВСУ) укомплектованы лишь наполовину. Однако, по словам экспертов, мобилизация уже создала огромный дефицит рабочей силы, а дополнительный призыв сотен тысяч человек приведет к частичному параличу экономики. ТАСС собрал актуальную информацию о событиях на Украине и вокруг нее. Вооруженные силы РФ за прошедшие сутки заняли более выгодные рубежи и позиции и отразили девять атак и контратак ВСУ на авдеевском направлении, три атаки на купянском направлении и три атаки на запорожском направлении, сообщили в Минобороны РФ. Потери противника за сутки на всех направлениях, включая донецкое, южнодонецкое и херсонское, составили до 840 бойцов. Российские средст

**почистим от стоп-слов**

In [6]:
cleared_docs = [[token for token in text if token not in stopword_set] for text in tqdm.tqdm(docs)]

100%|██████████| 2396/2396 [00:00<00:00, 10074.03it/s]


**лемматайзер**

In [7]:
!pip install pymorphy2==0.8
import pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 39.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=09806d2c18751d664911a80921aab3d487590180b31db7782958da56480c4914
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [8]:
lemmatizer = pymorphy2.MorphAnalyzer()

In [9]:
lemmatizer_cache = {}

def lemmatize(token):
    if lemmatizer.word_is_known(token):
        if token not in lemmatizer_cache:
            lemmatizer_cache[token] = lemmatizer.parse(token)[0].normal_form
        return lemmatizer_cache[token]
    return token

lemmatized_cleared_docs = [[lemmatize(token) for token in text] for text in tqdm.tqdm(cleared_docs)]

100%|██████████| 2396/2396 [01:06<00:00, 36.19it/s]


In [10]:
print(lemmatized_cleared_docs[0])

['президент', 'украина', 'владимир', 'зеленский', 'смочь', 'объяснить', 'заключаться', 'план', 'б', 'случай', 'вашингтон', 'предоставить', 'киев', 'новый', 'пакет', 'помощь', 'украинский', 'власть', 'выбор', 'проводить', 'проводить', 'мобилизация', 'часть', 'подразделение', 'вооружённый', 'сила', 'украина', 'всу', 'укомплектовать', 'лишь', 'наполовину', 'однако', 'слово', 'эксперт', 'мобилизация', 'создать', 'огромный', 'дефицит', 'рабочий', 'сила', 'дополнительный', 'призыв', 'сотня', 'тысяча', 'человек', 'привести', 'частичный', 'паралич', 'экономика', 'тасс', 'собрать', 'актуальный', 'информация', 'событие', 'украина', 'вокруг', 'вооружённый', 'сила', 'рф', 'прошедшее', 'сутки', 'занять', 'выгодный', 'рубеж', 'позиция', 'отразить', 'девять', 'атака', 'контратака', 'всу', 'авдеевский', 'направление', 'атака', 'купянск', 'направление', 'атака', 'запорожский', 'направление', 'сообщить', 'минобороны', 'рф', 'потеря', 'противник', 'сутки', 'направление', 'включая', 'донецкий', 'южнодонец

**морфологический словарь**

In [11]:
!gdown 1x5XFECfvU4JUAVgY0NeX5iL1PksRd4-6

Downloading...
From: https://drive.google.com/uc?id=1x5XFECfvU4JUAVgY0NeX5iL1PksRd4-6
To: /content/morph_dict.json
100% 10.3M/10.3M [00:00<00:00, 47.2MB/s]


In [12]:
with open('morph_dict.json') as file:
    morph_dict = json.load(file)

print(morph_dict['лукашенко'])
print(morph_dict['учёный'])

['лукашенко']
['ученым', 'ученая', 'ученые', 'учеными', 'ученой', 'ученому', 'ученый', 'ученом', 'ученого', 'ученых']


**word2vec**

In [ ]:
%%time
model = Word2Vec(lemmatized_cleared_docs, vector_size=100, window=5, min_count=5, sg=0)
model.wv.most_similar('учёный')

CPU times: user 22.9 s, sys: 119 ms, total: 23 s
Wall time: 15 s


[('профессия', 0.92118901014328),
 ('исследователь', 0.8991945385932922),
 ('знание', 0.8961362838745117),
 ('привлекать', 0.8951799273490906),
 ('семейный', 0.8791525959968567),
 ('показывать', 0.8791041970252991),
 ('проходная', 0.8711189031600952),
 ('молодёжь', 0.8709526062011719),
 ('отличаться', 0.8691685199737549),
 ('школьник', 0.8656061887741089)]

In [13]:
# model.save('word2vec-tass.ru-100')
!gdown 1i8karrg2YMmwwA4J-waQaS_o1jqIGRbT

Downloading...
From: https://drive.google.com/uc?id=1i8karrg2YMmwwA4J-waQaS_o1jqIGRbT
To: /content/word2vec-tass.ru-100
100% 14.1M/14.1M [00:00<00:00, 68.6MB/s]


In [14]:
model = Word2Vec.load('word2vec-tass.ru-100')
model.wv.most_similar('учёный')

[('профессия', 0.9160180687904358),
 ('исследователь', 0.9027774333953857),
 ('педагог', 0.9012601375579834),
 ('преподаватель', 0.8916919827461243),
 ('молодёжь', 0.8879184722900391),
 ('семейный', 0.8871046900749207),
 ('знание', 0.8864253163337708),
 ('отличаться', 0.8773655891418457),
 ('привлекать', 0.8752392530441284),
 ('учиться', 0.8695785999298096)]

**TRIE**

In [15]:
class TrieNode:
    def __init__(self, label: str, key: Optional[str] = None):
        self.children = [None] * 35     # не 33, потому что 'я' --> 'ѐ' --> 'ё'
        self.is_terminal = False
        self.reverse_index = {}


class Trie:
    def __init__(self):
        self.root = TrieNode(label='')

    def insert(self, key, doc_index, word_index):
        if not key:
            return None

        search_result = self.search(key)

        # если слово есть
        if search_result[0]:
            current_node = search_result[1]

            # если слово есть, но нет его документа --> добавляем к слову новый документ и индекс
            if doc_index not in current_node.reverse_index:
                current_node.reverse_index[doc_index] = [word_index]

            # если слово есть и есть его документ --> добавляем к документу новый индекс
            else:
                current_node.reverse_index[doc_index].append(word_index)
            return None


        # если слова нет --> добавляем слово с его индексом документа и индексом в документе
        current_node = self.root
        for char in key:
            index = ord(char) - ord('а')
            if not current_node.children[index]:
                current_node.children[index] = TrieNode(label=char)
            current_node = current_node.children[index]
        current_node.is_terminal = True
        current_node.key = key
        current_node.reverse_index[doc_index] = [word_index]


    def search(self, key):
        if not key:
            return None

        current_node = self.root

        for char in key:
            index = ord(char) - ord('а')
            if not current_node.children[index]:
                return False, ''
            current_node = current_node.children[index]

        return current_node.is_terminal, current_node

**префиксное дерево на текстах, очищенных от стоп-слов**

In [16]:
tass_trie = Trie()

for i in tqdm.tqdm(range(len(cleared_docs))):
    for j in range(len(cleared_docs[i])):
        tass_trie.insert(cleared_docs[i][j], i, j)

100%|██████████| 2396/2396 [00:12<00:00, 184.72it/s]


In [17]:
def _search(trie: Trie, morph_dict, w2v_model, *words, k) -> dict:
    """
    word --> lemma --> vector --> similars --> morph_dict = {'similar': [word forms]}
    word form search in the Trie --> {word form: TrieNode}
    """
    dict_result = {}

    for word in words:
        word = word.lower()
        lemma = lemmatizer.parse(word)[0].normal_form
        try:
            vector_lemma = model.wv[lemma]
        except KeyError:
            continue
        similars = model.wv.similar_by_vector(vector_lemma, topn=k)
        for elem in similars:
            try:
                tokens = morph_dict[elem[0]]
            except KeyError:
                continue
            score = elem[1]
            for token in tokens:
                result = trie.search(token)
                if result[0]:
                    dict_result[(token, score)] = result[1]
                else:
                    print('there is no such word in the database')

    return dict_result


def search(trie: Trie, morph_dict, w2v_model, *words, k=5) -> None:
    """
    вызывает функцию _search()
    отрезает k релевантных документов, печатает сниппеты из текстов
    """

    dict_result = _search(trie, morph_dict, w2v_model, *words, k=k)

    print("################################################")
    print("\n=========================================\n")
    for word in dict_result:
        indices = dict_result[word].reverse_index

        if len(indices) < k:
            k = len(indices)
        sorted_indices = sorted(indices.items(), key=lambda item: -len(item[1]))[:k]

        print(f'the word "{word}" appears in:')
        for key in sorted_indices:
            print(f'TITLE: {articles.title[key[0]]}')
            indices_in_doc = key[1]
            for index in indices_in_doc:
                if 0 <= index + 2 < len(cleared_docs[key[0]]) and 0 <= index - 2 < len(cleared_docs[key[0]]):
                    print(f'SNIPPET: "...{cleared_docs[key[0]][index-2]} {cleared_docs[key[0]][index-1]} {cleared_docs[key[0]][index]} {cleared_docs[key[0]][index+1]} {cleared_docs[key[0]][index+2]}..."')
                elif 0 <= index - 2 < len(cleared_docs[key[0]]):
                    print(f'SNIPPET: "...{cleared_docs[key[0]][index-2]} {cleared_docs[key[0]][index-1]} {cleared_docs[key[0]][index]}..."')
                else:
                    print(f'SNIPPET: "...{cleared_docs[key[0]][index]} {cleared_docs[key[0]][index+1]} {cleared_docs[key[0]][index+2]}..."')
            print("\n=========================================\n")
        print("################################################\n")

In [19]:
search(tass_trie, morph_dict, model, 'люди', k=3)

################################################


the word "('человеке', 1.0)" appears in:
TITLE: В поисках человечности. Почему история с Твиксом стала известной и чему она нас научила
SNIPPET: "...жив человек человеке доброе светлое..."


TITLE: Влюбленный инженер, трактирщик и барон: семь ярких ролей Юрия Соломина
SNIPPET: "...лучший фильм человеке природе кстати..."


TITLE: Новатор и друг Яшина. Умер Франц Беккенбауэр
SNIPPET: "...нем замечательном человеке сказал кавазашвили..."


################################################

the word "('люди', 1.0)" appears in:
TITLE: Дмитрий Медведев: ненавижу врагов России
SNIPPET: "...какие круги люди видели значит..."
SNIPPET: "...форме такие люди количество расти..."
SNIPPET: "...войска сами люди населяющие земли..."
SNIPPET: "...совершенно другие люди которые осознают..."
SNIPPET: "...украина такие люди появятся ними..."
SNIPPET: "...города какие люди живут каком..."
SNIPPET: "...остальные это люди которых своей..."
SNIPPET: "...разго